# Byte-Pair Encoding tokenization

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [ ]:
# Install required packages for building a Byte-Pair Encoding (BPE) tokenizer from scratch
# - datasets: For loading and processing text datasets
# - evaluate: For model evaluation metrics  
# - transformers[sentencepiece]: Core library with SentencePiece support
!uv pip install datasets evaluate transformers[sentencepiece]

In [ ]:
# Define a small training corpus to demonstrate BPE algorithm step-by-step
# This corpus will be used to learn which character pairs to merge most frequently
# BPE starts with individual characters and iteratively merges the most common pairs
corpus = [
    "This is the Hugging Face Course.",
    "This chapter is about tokenization.",
    "This section shows several tokenizer algorithms.",
    "Hopefully, you will be able to understand how they are trained and generate tokens.",
]

In [ ]:
# Load GPT-2 tokenizer to understand its pre-tokenization step
# GPT-2 uses BPE and we'll replicate its approach
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [ ]:
# Step 1: Pre-tokenize the corpus and count word frequencies
# Pre-tokenization splits text into words, preserving spaces as 'Ġ' tokens
# This frequency counting helps BPE prioritize merges for common word patterns
from collections import defaultdict

word_freqs = defaultdict(int)

for text in corpus:
    # Pre-tokenize each sentence using GPT-2's method
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, offset in words_with_offsets]
    # Count frequency of each pre-tokenized word
    for word in new_words:
        word_freqs[word] += 1

print(word_freqs)

In [ ]:
# Step 2: Extract the base alphabet from all words
# The alphabet consists of all unique characters that appear in the corpus
# This forms the initial vocabulary before any merges
alphabet = []

for word in word_freqs.keys():
    for letter in word:
        if letter not in alphabet:
            alphabet.append(letter)
alphabet.sort()

print(alphabet)

In [ ]:
# Step 3: Initialize vocabulary with special tokens and base alphabet
# "<|endoftext|>" is GPT-2's special token for end of document
# The base vocabulary starts with special tokens plus all individual characters
vocab = ["<|endoftext|>"] + alphabet.copy()

In [ ]:
# Step 4: Split each word into individual characters for BPE training
# This creates the initial splits that BPE will iteratively merge
# Each word becomes a list of characters that can be progressively combined
splits = {word: [c for c in word] for word in word_freqs.keys()}

In [ ]:
# Step 5: Function to compute frequencies of adjacent character pairs
# BPE works by finding the most frequent adjacent pair and merging it
# This function counts how often each pair appears across all words (weighted by word frequency)
def compute_pair_freqs(splits):
    pair_freqs = defaultdict(int)
    for word, freq in word_freqs.items():
        split = splits[word]
        if len(split) == 1:
            continue  # Skip single-character words
        # Count each adjacent pair in the word
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            pair_freqs[pair] += freq  # Weight by word frequency
    return pair_freqs

In [ ]:
# Step 6: Compute initial pair frequencies to see what BPE will merge first
# This shows the character pairs that appear most frequently in our corpus
pair_freqs = compute_pair_freqs(splits)

# Display the first few pairs and their frequencies
for i, key in enumerate(pair_freqs.keys()):
    print(f"{key}: {pair_freqs[key]}")
    if i >= 5:
        break

In [ ]:
# Step 7: Find the most frequent pair to merge first
# BPE always merges the pair with the highest frequency
# This greedy approach builds subwords from the most common patterns
best_pair = ""
max_freq = None

for pair, freq in pair_freqs.items():
    if max_freq is None or max_freq < freq:
        best_pair = pair
        max_freq = freq

print(best_pair, max_freq)

In [ ]:
# Step 8: Record the first merge and add new token to vocabulary
# Keep track of all merges for later tokenization
# Add the merged token to our growing vocabulary
merges = {("Ġ", "t"): "Ġt"}
vocab.append("Ġt")

In [ ]:
# Step 9: Function to apply a merge to all word splits
# When we decide to merge a pair, we need to update all word splits
# This function finds all instances of the pair and combines them
def merge_pair(a, b, splits):
    for word in word_freqs:
        split = splits[word]
        if len(split) == 1:
            continue  # Skip single-character words

        i = 0
        while i < len(split) - 1:
            # If we find the pair to merge, combine them
            if split[i] == a and split[i + 1] == b:
                split = split[:i] + [a + b] + split[i + 2 :]
            else:
                i += 1  # Move to next position only if no merge occurred
        splits[word] = split
    return splits

In [ ]:
# Step 10: Apply the first merge and see the result
# This demonstrates how "Ġ" + "t" becomes "Ġt" in all affected words
splits = merge_pair("Ġ", "t", splits)
print(splits["Ġtrained"])  # Example of how the merge affected this word

In [ ]:
# Step 11: Continue BPE training until we reach our target vocabulary size
# This is the main BPE training loop that iteratively merges the most frequent pairs
vocab_size = 50

while len(vocab) < vocab_size:
    # Find the most frequent pair in current splits
    pair_freqs = compute_pair_freqs(splits)
    best_pair = ""
    max_freq = None
    for pair, freq in pair_freqs.items():
        if max_freq is None or max_freq < freq:
            best_pair = pair
            max_freq = freq
    
    # Apply the merge and update our vocabulary and merge rules
    splits = merge_pair(*best_pair, splits)
    merges[best_pair] = best_pair[0] + best_pair[1]
    vocab.append(best_pair[0] + best_pair[1])

In [ ]:
# Step 12: Examine the learned merge rules
# These rules define the order and mapping of character pair merges
# Each merge rule maps a pair of tokens to their combined form
print(merges)

In [ ]:
# Step 13: Display the final vocabulary after BPE training
# The vocabulary now contains the original characters plus learned subwords
# Common subwords like "This", "Ġtoken", and "Ġthe" were automatically discovered
print(vocab)

In [ ]:
# Step 14: Implement tokenization function using our learned BPE rules
# This function applies all merge rules in order to tokenize new text
def tokenize(text):
    # First, pre-tokenize the text using the same method as training
    pre_tokenize_result = tokenizer._tokenizer.pre_tokenizer.pre_tokenize_str(text)
    pre_tokenized_text = [word for word, offset in pre_tokenize_result]
    
    # Split each word into characters
    splits = [[l for l in word] for word in pre_tokenized_text]
    
    # Apply all learned merges in the order they were learned
    for pair, merge in merges.items():
        for idx, split in enumerate(splits):
            i = 0
            while i < len(split) - 1:
                if split[i] == pair[0] and split[i + 1] == pair[1]:
                    split = split[:i] + [merge] + split[i + 2 :]
                else:
                    i += 1
            splits[idx] = split

    # Flatten the list of lists into a single list of tokens
    return sum(splits, [])

In [ ]:
# Step 15: Test our BPE tokenizer on new text
# Notice how it handles words that were in training vs. new words
# "token" was seen in training, but "not" was not, so it gets split into characters
tokenize("This is not a token.")